<div style="border: 2px solid #575757; padding: 10px; border-radius: 5px; background-color: #e1e1e1; color: black; text-align: center;">
  <h1 style="margin: 0;">Life cycle impact assessment results</h1>
</div>

<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Libraries importation</h2>
</div>

**Import required libraries:**

In [1]:
import os
import re
import glob
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import kendalltau
import joblib
from pathlib import Path
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib.font_manager import FontProperties
import seaborn as sns
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output, State

<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Metadata definition</h2>
</div>

**Define the metadata of LCIA methods to index the excel results files:**

In [2]:

# Define metadata for all Excel files
metadata = [
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-acidification-accumulated-exceedance-AE.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-acidification-accumulated-exceedance-AE_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-acidification-accumulated-exceedance-AE.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Acidification',
        'Impact Indicator': 'Accumulated Exceedance (Acidification)',
        'Short Name': 'AE-A',
        'Unit': 'mol H+ eq',
        'Robustness': 'II'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-climate-change-global-warming-potential-GWP100.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-climate-change-global-warming-potential-GWP100_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-climate-change-global-warming-potential-GWP100.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Climate Change',
        'Impact Indicator': 'Global Warming Potential (Climate Change)',
        'Short Name': 'GWP100',
        'Unit': 'kg CO2 eq',
        'Robustness': 'I'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-ecotoxicity-freshwater-comparative-toxic-unit-for-ecosystems-CTUe.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-ecotoxicity-freshwater-comparative-toxic-unit-for-ecosystems-CTUe_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-ecotoxicity-freshwater-comparative-toxic-unit-for-ecosystems-CTUe.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Ecotoxicity',
        'Impact Indicator': 'Ecotoxicity (Freshwater)',
        'Short Name': 'ECOTOX-FW',
        'Unit': 'CTUe',
        'Robustness': 'II/III'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-energy-resources-non-renewable-abiotic-depletion-potential-ADP-fossil-fuels.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-energy-resources-non-renewable-abiotic-depletion-potential-ADP-fossil-fuels_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-energy-resources-non-renewable-abiotic-depletion-potential-ADP-fossil-fuels.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Energy Resources: non-renewable',
        'Impact Indicator': 'Abiotic Depletion Potential (Fossil Fuels)',
        'Short Name': 'ADP-FF',
        'Unit': 'MJ',
        'Robustness': 'III'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-eutrophication-freshwater-fraction-of-nutrients-reaching-freshwater-end-compartment-P.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-eutrophication-freshwater-fraction-of-nutrients-reaching-freshwater-end-compartment-P_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-eutrophication-freshwater-fraction-of-nutrients-reaching-freshwater-end-compartment-P.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Eutrophication: freshwater',
        'Impact Indicator': 'Fraction of nutrients reaching compartment (Freshwater Eutrophication)',
        'Short Name': 'EUT-FW',
        'Unit': 'kg P eq',
        'Robustness': 'II'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-eutrophication-marine-fraction-of-nutrients-reaching-marine-end-compartment-N.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-eutrophication-marine-fraction-of-nutrients-reaching-marine-end-compartment-N_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-eutrophication-marine-fraction-of-nutrients-reaching-marine-end-compartment-N.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Eutrophication: marine',
        'Impact Indicator': 'Fraction of nutrients reaching compartment (Marine Eutrophication)',
        'Short Name': 'EUT-M',
        'Unit': 'kg N eq',
        'Robustness': 'II'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-eutrophication-terrestrial-accumulated-exceedance-AE.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-eutrophication-terrestrial-accumulated-exceedance-AE_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-eutrophication-terrestrial-accumulated-exceedance-AE.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Eutrophication: terrestrial',
        'Impact Indicator': 'Fraction of nutrients reaching compartment (Terrestrial Eutrophication)',
        'Short Name': 'EUT-T',
        'Unit': 'mol N eq',
        'Robustness': 'II'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-human-toxicity-carcinogenic-comparative-toxic-unit-for-human-CTUh.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-human-toxicity-carcinogenic-comparative-toxic-unit-for-human-CTUh_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-human-toxicity-carcinogenic-comparative-toxic-unit-for-human-CTUh.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Toxicity: human health',
        'Impact Indicator': 'Human Toxicity (Carcinogenic)',
        'Short Name': 'HTOX-C',
        'Unit': 'CTUh',
        'Robustness': 'II/III'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-human-toxicity-non-carcinogenic-comparative-toxic-unit-for-human-CTUh.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-human-toxicity-non-carcinogenic-comparative-toxic-unit-for-human-CTUh_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-human-toxicity-non-carcinogenic-comparative-toxic-unit-for-human-CTUh.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Toxicity: human health',
        'Impact Indicator': 'Human Toxicity (Non-Carcinogenic)',
        'Short Name': 'HTOX-NC',
        'Unit': 'CTUh',
        'Robustness': 'II/III'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-ionising-radiation-human-health-human-exposure-efficiency-relative-to-u235.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-ionising-radiation-human-health-human-exposure-efficiency-relative-to-u235_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-ionising-radiation-human-health-human-exposure-efficiency-relative-to-u235.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Ionising Radiation: human health',
        'Impact Indicator': 'Human Exposure Efficiency relative to U235 (Human Health)',
        'Short Name': 'IR',
        'Unit': 'kBq U235 eq',
        'Robustness': 'II'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-land-use-soil-quality-index.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-land-use-soil-quality-index_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-land-use-soil-quality-index.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Land Use',
        'Impact Indicator': 'Soil Quality Index (Land Use)',
        'Short Name': 'LU-SQI',
        'Unit': 'dimensionless',
        'Robustness': 'III'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-material-resources-metalsminerals-abiotic-depletion-potential-ADP-elements-ultimate-reserves.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-material-resources-metalsminerals-abiotic-depletion-potential-ADP-elements-ultimate-reserves_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-material-resources-metalsminerals-abiotic-depletion-potential-ADP-elements-ultimate-reserves.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Material resources: metals/minerals',
        'Impact Indicator': 'Abiotic Depletion Potential (Ultimate Reserves)',
        'Short Name': 'ADP-UR',
        'Unit': 'kg Sb eq',
        'Robustness': 'III'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-ozone-depletion-ozone-depletion-potential-ODP.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-ozone-depletion-ozone-depletion-potential-ODP_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-ozone-depletion-ozone-depletion-potential-ODP.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Ozone Depletion',
        'Impact Indicator': 'Ozone Depletion Potential',
        'Short Name': 'ODP',
        'Unit': 'kg CFC-11 eq',
        'Robustness': 'I'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-particulate-matter-formation-impact-on-human-health.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-particulate-matter-formation-impact-on-human-health_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-ozone-depletion-ozone-depletion-potential-ODP.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Particulate Matter: human health',
        'Impact Indicator': 'Particulate Matter Formation (Human Health)',
        'Short Name': 'PMF',
        'Unit': 'disease incidence',
        'Robustness': 'I'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-photochemical-oxidant-formation-human-health-tropospheric-ozone-concentration-increase.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-photochemical-oxidant-formation-human-health-tropospheric-ozone-concentration-increase_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-photochemical-oxidant-formation-human-health-tropospheric-ozone-concentration-increase.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Photochemical Oxidant Formation: human health',
        'Impact Indicator': 'Photochemical Oxidant Formation (Human Health)',
        'Short Name': 'POF',
        'Unit': 'kg NMVOC eq',
        'Robustness': 'II'
    },
    {
        'MC Filename': 'Static-LCA_Monte-Carlo-results_EF-v31-water-use-user-deprivation-potential-deprivation-weighted-water-consumption.xlsx',
        'CA_RF Filename': 'Static-LCA_First-Tier-contributions_EF-v31-water-use-user-deprivation-potential-deprivation-weighted-water-consumption_relative-share.xlsx',
        'GSA Filename': 'gsa_output_Static-LCA_1000_EF-v31-water-use-user-deprivation-potential-deprivation-weighted-water-consumption.xlsx',
        'Impact Method': 'EF v3.1',
        'Impact Category': 'Water Use',
        'Impact Indicator': 'User Deprivation Potential (Water Use)',
        'Short Name': 'UDP-WU',
        'Unit': 'm3 world eq deprived',
        'Robustness': 'III'
    },
]

meta_df = pd.DataFrame(metadata)
display(meta_df)


# Define Scenario → H2 storage and Scenario family mappings
h2_map = {
    f"SIM{i:02d}": val for i, val in zip(range(1,29),
        [1000,500,100,50,30,10,1,
         1000,500,100,50,30,10,1,
         1000,500,100,50,30,10,1,
         1000,500,100,50,30,10,1])
}
fam_map = {
    **{f"SIM{i:02d}": "PV + WT + BAT" for i in range(1,8)},
    **{f"SIM{i:02d}": "WT + BAT"       for i in range(8,15)},
    **{f"SIM{i:02d}": "PV + BAT"       for i in range(15,22)},
    **{f"SIM{i:02d}": "PV"             for i in range(22,29)},
}

,MC Filename,CA_RF Filename,GSA Filename,Impact Method,Impact Category,Impact Indicator,Short Name,Unit,Robustness
0,Static-LCA_Monte-Carlo-results_EF-v31-acidific...,Static-LCA_First-Tier-contributions_EF-v31-aci...,gsa_output_Static-LCA_1000_EF-v31-acidificatio...,EF v3.1,Acidification,Accumulated Exceedance (Acidification),AE-A,mol H+ eq,II
1,Static-LCA_Monte-Carlo-results_EF-v31-climate-...,Static-LCA_First-Tier-contributions_EF-v31-cli...,gsa_output_Static-LCA_1000_EF-v31-climate-chan...,EF v3.1,Climate Change,Global Warming Potential (Climate Change),GWP100,kg CO2 eq,I
2,Static-LCA_Monte-Carlo-results_EF-v31-ecotoxic...,Static-LCA_First-Tier-contributions_EF-v31-eco...,gsa_output_Static-LCA_1000_EF-v31-ecotoxicity-...,EF v3.1,Ecotoxicity,Ecotoxicity (Freshwater),ECOTOX-FW,CTUe,II/III
3,Static-LCA_Monte-Carlo-results_EF-v31-energy-r...,Static-LCA_First-Tier-contributions_EF-v31-ene...,gsa_output_Static-LCA_1000_EF-v31-energy-resou...,EF v3.1,Energy Resources: non-renewable,Abiotic Depletion Potential (Fossil Fuels),ADP-FF,MJ,III
4,Static-LCA_Monte-Carlo-results_EF-v31-eutrophi...,Static-LCA_First-Tier-contributions_EF-v31-eut...,gsa_output_Static-LCA_1000_EF-v31-eutrophicati...,EF v3.1,Eutrophication: freshwater,Fraction of nutrients reaching compartment (Fr...,EUT-FW,kg P eq,II
5,Static-LCA_Monte-Carlo-results_EF-v31-eutrophi...,Static-LCA_First-Tier-contributions_EF-v31-eut...,gsa_output_Static-LCA_1000_EF-v31-eutrophicati...,EF v3.1,Eutrophication: marine,Fraction of nutrients reaching compartment (Ma...,EUT-M,kg N eq,II
6,Static-LCA_Monte-Carlo-results_EF-v31-eutrophi...,Static-LCA_First-Tier-contributions_EF-v31-eut...,gsa_output_Static-LCA_1000_EF-v31-eutrophicati...,EF v3.1,Eutrophication: terrestrial,Fraction of nutrients reaching compartment (Te...,EUT-T,mol N eq,II
7,Static-LCA_Monte-Carlo-results_EF-v31-human-to...,Static-LCA_First-Tier-contributions_EF-v31-hum...,gsa_output_Static-LCA_1000_EF-v31-human-toxici...,EF v3.1,Toxicity: human health,Human Toxicity (Carcinogenic),HTOX-C,CTUh,II/III
8,Static-LCA_Monte-Carlo-results_EF-v31-human-to...,Static-LCA_First-Tier-contributions_EF-v31-hum...,gsa_output_Static-LCA_1000_EF-v31-human-toxici...,EF v3.1,Toxicity: human health,Human Toxicity (Non-Carcinogenic),HTOX-NC,CTUh,II/III
9,Static-LCA_Monte-Carlo-results_EF-v31-ionising...,Static-LCA_First-Tier-contributions_EF-v31-ion...,gsa_output_Static-LCA_1000_EF-v31-ionising-rad...,EF v3.1,Ionising Radiation: human health,Human Exposure Efficiency relative to U235 (Hu...,IR,kBq U235 eq,II


<div style="border: 2px solid rgba(0, 158, 115, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 158, 115, 0.3); color: black; text-align: center;">
  <h2 style="margin: 0;">Monte Carlo (MC) data importation</h2>
</div>

**Read and extract the LCIA results data from the Monte Carlo simulations:**

**Optional if you already have the pickle files**

In [ ]:
# Base directory containing subfolders
base_dir = Path.cwd() / "MC"

# Create a directory for pickles
pickles_dir = Path.cwd() / "pickles"
pickles_dir.mkdir(exist_ok=True)

# Loop over each subfolder
for sub in base_dir.iterdir():
    if not sub.is_dir():
        continue

    df_list = []
    for xlsx in sub.glob("*.xlsx"):
        # metadata lookup: exact match on 'MC Filename'
        row = meta_df[meta_df['MC Filename'] == xlsx.name]
        if row.empty:
            raise ValueError(f"No metadata for file: {xlsx.name}")

        md = row.iloc[0].to_dict()

        # Read Excel and drop the first (unnamed) column
        tmp = pd.read_excel(xlsx, sheet_name=0)
        tmp = tmp.iloc[:, 1:]  # drop the very first unnamed column

        # Melt SIM columns, extract SIM code
        long = (
            tmp
            .melt(var_name='raw_col', value_name='value')
            .dropna(subset=['value'])
            .assign(
                Scenario=lambda df: df['raw_col'].str.extract(r'(SIM\d{2})')[0]
            )
            .drop(columns=['raw_col'])
        )

        # Map H2 storage & Scenario family
        long['H2 storage']      = long['Scenario'].map(h2_map)
        long['Scenario family'] = long['Scenario'].map(fam_map)

        # Add metadata columns
        for col in ['Impact Method', 'Impact Category', 'Impact Indicator',
                    'Short Name', 'Unit', 'Robustness']:
            long[col] = md[col]

        df_list.append(long)

    # Concatenate all results
    result = pd.concat(df_list, ignore_index=True)

    # Save as pickle
    out_path = pickles_dir / f"{sub.name}_MC.pkl"
    joblib.dump(result, out_path)

    # Assign to global variable
    globals()[sub.name] = result
    print(f"Folder '{sub.name}': {len(result)} rows → saved to {out_path}")

    display(result)


<div style="border: 2px solid rgba(0, 158, 115, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 158, 115, 0.3); color: black; text-align: center;">
  <h2 style="margin: 0;">MC data visualization</h2>
</div>

**Display interactive widget for lineplots:**

**Only selected EF v3.1 impact methods as default**:
- That can be changed in the "Families & Indicators" menu
- The "Axis Options" menu allow different scalings for display

**Help for interpretation:**
- Within each family, progression follows decreasing H2 storage capacity in m3 (1000, 500, 100, 50, 30, 10, 1)
- The rows represent the environmental category tested
- Columns represent the impact boundary
- The solid lines represent the median values of the Monte Carlo simulations
- The dotted lines delimit the quartiles ± 25% around the median values

In [3]:
############## LOAD AND COMBINE DATA ##############

# Load and label manually listed "_MC.pkl" files
pickle_files = [
    "pickles/Load_MC.pkl",
    "pickles/Load+Excess_MC.pkl",
    "pickles/Load+Excess-GridDE_MC.pkl"
]

df_list = []
for fpath in pickle_files:
    tmp = joblib.load(fpath)
    key = os.path.splitext(os.path.basename(fpath))[0].replace('_MC', '')
    tmp['Source'] = key

    # Extract scenario number
    tmp["Scenario Number"] = tmp["Scenario"].str.extract(r'(\d+)').astype(int)

    # Assign scenario family based on Scenario Number
    conditions = [
        tmp["Scenario Number"] <= 7,
        (tmp["Scenario Number"] >= 8) & (tmp["Scenario Number"] <= 14),
        (tmp["Scenario Number"] >= 15) & (tmp["Scenario Number"] <= 21),
        tmp["Scenario Number"] >= 22
    ]
    families = ["PV + WT + BAT", "WT + BAT", "PV + BAT", "PV"]
    tmp["Family"] = np.select(conditions, families, default="")

    df_list.append(tmp)

# Combine all data
df = pd.concat(df_list, ignore_index=True)

############## SETUP ##############

# Setup of metadata and other parameters
default_indicators = [
    "Global Warming Potential (Climate Change)",
    "Ecotoxicity (Freshwater)",
    "Human Toxicity (Carcinogenic)",
    "Abiotic Depletion Potential (Ultimate Reserves)",
    "User Deprivation Potential (Water Use)"
]
all_indicators = sorted(df["Impact Indicator"].unique())
selected_defaults = [ind for ind in default_indicators if ind in all_indicators]
all_families = sorted(df["Family"].unique())
all_sources = sorted(df["Source"].unique())

# Clean Source Titles
def format_title(text):
    t = re.sub(r"\s*[+]\s*", " + ", text)
    t = re.sub(r"\s*[-]\s*", " - ", t)
    t = t.replace('GridDE', 'Grid (DE)')
    return t.strip()
formatted_titles = [format_title(c) for c in all_sources]

# Color palette per Scenario Family
color_palette = sns.color_palette("colorblind", n_colors=len(set(df["Family"])))
family_color_dict = dict(zip(["PV + WT + BAT", "WT + BAT", "PV + BAT", "PV"], color_palette))

############## WIDGETS ##############
family_sel = widgets.SelectMultiple(
    options=all_families,
    value=tuple(all_families),
    description="Scenario Families",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="350px", height="120px")
)
ind_sel = widgets.SelectMultiple(
    options=all_indicators,
    value=tuple(selected_defaults),
    description="Impact Indicators",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="350px", height="180px")
)

# New Axis Scale Widgets
ys = widgets.ToggleButtons(
    options=["linear", "log"],
    value="linear",
    description="Y-axis Scale",
    style={"description_width": "initial"}
)
force_row = widgets.ToggleButtons(
    options=["adaptative", "forced"],
    value="adaptative",
    description="Force row-wise Y-scale",
    style={"description_width": "initial"}
)
savefig = widgets.ToggleButtons(
    options=["unsaved", "saved"],
    value="unsaved",
    description="Save Figure",
    style={"description_width": "initial"}
)

############## PLOT FUNCTION ##############
def update_plot(fams, inds, yscale, force_row_scale, savefig):
    if not fams or not inds:
        print("Select at least one family and one indicator.")
        return
    sub = df[df["Family"].isin(fams) & df["Impact Indicator"].isin(inds)].copy()

    # Control the order
    sub["Impact Indicator"] = pd.Categorical(sub["Impact Indicator"], categories=default_indicators, ordered=True)
    sub = sub.sort_values(by="Impact Indicator")

    g = sns.FacetGrid(
        sub,
        row="Impact Indicator",
        row_order=default_indicators,
        col="Source",
        col_order=all_sources,
        height=3,
        aspect=1.4,
        sharey=False,
        sharex=True
    )

    def plot_lines(data, ax, ymin=None, ymax=None):
        grouped = data.groupby(["Scenario Number", "Family"])

        summary = grouped["value"].agg([
            'median',
            'min',
            'max',
            lambda x: np.percentile(x, 25),
            lambda x: np.percentile(x, 75)
        ]).reset_index()
        summary.columns = ["Scenario Number", "Family", "Median", "Min", "Max", "Q1", "Q3"]

        for fam in data["Family"].unique():
            fam_data = summary[summary["Family"] == fam]
            if fam_data.empty:
                continue

            # Fill Min → Q1
            ax.fill_between(
                fam_data["Scenario Number"],
                fam_data["Min"],
                fam_data["Q1"],
                color=family_color_dict[fam],
                alpha=0.25  # outside region
            )

            # Fill Q1 → Q3
            ax.fill_between(
                fam_data["Scenario Number"],
                fam_data["Q1"],
                fam_data["Q3"],
                color=family_color_dict[fam],
                alpha=0.5,  # inside region between dotted lines
                label=fam if fam not in ax.get_legend_handles_labels()[1] else ""
            )

            # Fill Q3 → Max
            ax.fill_between(
                fam_data["Scenario Number"],
                fam_data["Q3"],
                fam_data["Max"],
                color=family_color_dict[fam],
                alpha=0.25  # outside region
            )

            ax.plot(
                fam_data["Scenario Number"],
                fam_data["Q1"],
                linestyle='--',
                color=family_color_dict[fam],
                linewidth=1
            )
            ax.plot(
                fam_data["Scenario Number"],
                fam_data["Q3"],
                linestyle='--',
                color=family_color_dict[fam],
                linewidth=1
            )
            ax.plot(
                fam_data["Scenario Number"],
                fam_data["Median"],
                linestyle='-',
                color=family_color_dict[fam],
                linewidth=2
            )

        # Dynamic y-axis label: Short Name + Unit
        indicator_name = data['Impact Indicator'].iloc[0]
        tmp = sub[sub["Impact Indicator"] == indicator_name]
        if not tmp.empty:
            short_name = tmp["Short Name"].iloc[0]
            unit = tmp["Unit"].iloc[0]
            ax.set_ylabel(f"{short_name} ({unit})")

        # Title formatting
        title = indicator_name
        if "(" in title:
            before_paren, after_paren = title.split("(", 1)
            formatted_title = f"{before_paren.strip()}\n({after_paren.strip()}"
        else:
            formatted_title = title
        ax.set_title(formatted_title, pad=5, fontsize=10)

        # Scientific y-axis format
        fmt = ScalarFormatter(useMathText=True)
        fmt.set_scientific(True)
        fmt.set_powerlimits((-4, 4))
        ax.yaxis.set_major_formatter(fmt)

        # Set log scale if requested
        if yscale == "log":
            ax.set_yscale('log')

        # Set forced limits if requested
        if ymin is not None and ymax is not None:
            ax.set_ylim(ymin, ymax)

        # Suppress x-ticks
        ax.set_xticks([])
        ax.set_xticklabels([])
        ax.tick_params(axis='x', which='both', length=0)
        ax.minorticks_off()

    # Compute forced row y-limits if needed
    ylims = {}
    if force_row_scale == "forced":
        for ind in inds:
            vals = sub[sub["Impact Indicator"] == ind]["value"]
            ylims[ind] = (vals.min(), vals.max())

    # Draw plots
    for ax, ((ind, src), data) in zip(g.axes.flat, sub.groupby(['Impact Indicator', 'Source'], observed=False)):
        ymin, ymax = ylims.get(ind, (None, None)) if force_row_scale == "forced" else (None, None)
        plot_lines(data, ax, ymin, ymax)

    # Titles above each column
    for i, ind in enumerate(inds):
        for j, src in enumerate(all_sources):
            ax = g.axes[i, j]
            if i == 0:
                ax.text(0.5, 1.25, formatted_titles[j], transform=ax.transAxes,
                        ha='center', va='bottom', fontweight='bold')

    # Shared X Axis family labeling
    family_midpoints = [np.mean([1, 7]), np.mean([8, 14]), np.mean([15, 21]), np.mean([22, 28])]
    family_labels = ["PV + WT + BAT", "WT + BAT", "PV + BAT", "PV"]
    for ax in g.axes.flat:
        ax.set_xticks(family_midpoints)
        ax.set_xticklabels(family_labels, rotation=45, ha='right')
        ax.set_xlabel("Scenario Family", fontweight='bold')

    plt.tight_layout()

    # Save figure if requested
    if savefig == "saved":
        fig = plt.gcf()
        fig.savefig("LCIA_MC.png", dpi=600, bbox_inches="tight")
        fig.savefig("LCIA_MC.pdf", bbox_inches="tight")
        print("Figure saved as 'LCIA_MC.png' and 'LCIA_MC.pdf'.")

    plt.show()

############## BUILD USER INTERFACE ##############
filter_panel = widgets.VBox([
    widgets.HTML("<b>Filter Options</b>"),
    family_sel,
    ind_sel
])
axis_panel = widgets.VBox([
    widgets.HTML("<b>Axis Scale Options</b>"),
    ys,
    force_row
])
save_panel = widgets.VBox([
    widgets.HTML("<b>Save Figure</b>"),
    savefig
])
accordion = widgets.Accordion(children=[filter_panel, axis_panel, save_panel])
accordion.set_title(0, "Select Families & Indicators")
accordion.set_title(1, "Axis Scale Options")
accordion.set_title(2, "Save Figure")

display(accordion, widgets.interactive_output(
    update_plot,
    {"fams": family_sel, "inds": ind_sel, "yscale": ys, "force_row_scale": force_row, "savefig": savefig}
))


Accordion(children=(VBox(children=(HTML(value='<b>Filter Options</b>'), SelectMultiple(description='Scenario F…

Output()

<div style="border: 2px solid rgba(0, 158, 115, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 158, 115, 0.3); color: black; text-align: center;">
  <h2 style="margin: 0;">MC data statistics</h2>
</div>

**Display interactive widget for exploring data of contribution to impacts per Reference flow.**

**Filtering options:**
- Case
- Scenario
- Scenario family

In [2]:
############## LOAD AND LABEL MONTE CARLO DATA ##############
pickle_files = [
    "pickles/Load_MC.pkl",
    "pickles/Load+Excess_MC.pkl",
    "pickles/Load+Excess-GridDE_MC.pkl"
]

def format_title(key):
    t = re.sub(r"\s*[+]\s*", " + ", key)
    t = re.sub(r"\s*[-]\s*", " - ", t)
    t = t.replace('GridDE', 'Grid (DE)')
    return t.strip()

df_list = []
case_keys = []

for fpath in pickle_files:
    tmp = joblib.load(fpath)
    key = os.path.splitext(os.path.basename(fpath))[0].replace('_MC', '')
    tmp['Source'] = key
    tmp["Scenario Number"] = tmp["Scenario"].str.extract(r'(\d+)').astype(int)
    conditions = [
        tmp["Scenario Number"] <= 7,
        (tmp["Scenario Number"] >= 8) & (tmp["Scenario Number"] <= 14),
        (tmp["Scenario Number"] >= 15) & (tmp["Scenario Number"] <= 21),
        tmp["Scenario Number"] >= 22
    ]
    families = ["PV + WT + BAT", "WT + BAT", "PV + BAT", "PV"]
    tmp["Family"] = np.select(conditions, families, default="")
    df_list.append(tmp)
    case_keys.append(key)

df = pd.concat(df_list, ignore_index=True)

############## SETUP ##############
formatted_titles = [format_title(c) for c in case_keys]
case_map = dict(zip(formatted_titles, case_keys))

# Indicators
default_indicators = [
    "Global Warming Potential (Climate Change)",
    "Ecotoxicity (Freshwater)",
    "Human Toxicity (Carcinogenic)",
    "Abiotic Depletion Potential (Ultimate Reserves)",
    "User Deprivation Potential (Water Use)"
]
all_indicators = sorted(df["Impact Indicator"].unique())

############## WIDGETS ##############
case_dropdown = widgets.SelectMultiple(
    options=formatted_titles,
    description="Case(s):",
    layout=widgets.Layout(width='250px'),
    style={'description_width': 'initial'}
)

scenario_dropdown = widgets.SelectMultiple(
    options=sorted(df["Scenario"].unique()),
    description="Scenario(s):",
    layout=widgets.Layout(width='250px'),
    style={'description_width': 'initial'}
)

family_dropdown = widgets.SelectMultiple(
    options=sorted(df["Family"].dropna().unique()),
    description="Family(ies):",
    layout=widgets.Layout(width='250px'),
    style={'description_width': 'initial'}
)

indicator_selector = widgets.ToggleButtons(
    options=["Default Indicators", "All Indicators"],
    value="Default Indicators",
    description="Indicator Set:",
    style={'description_width': 'initial'}
)

# Output areas
output_df = widgets.Output()
output_stats = widgets.Output()

############## FILTER AND DISPLAY DATAFRAME ##############
def update_df(change=None):
    output_df.clear_output()
    output_stats.clear_output()

    selected_cases = [case_map[c] for c in case_dropdown.value]
    filtered = df[df["Source"].isin(selected_cases)]

    if scenario_dropdown.value:
        filtered = filtered[filtered["Scenario"].isin(scenario_dropdown.value)]

    if family_dropdown.value:
        filtered = filtered[filtered["Family"].isin(family_dropdown.value)]

    # Indicator filtering
    if indicator_selector.value == "Default Indicators":
        indicators_to_use = default_indicators
    else:
        indicators_to_use = all_indicators

    filtered = filtered[filtered["Impact Indicator"].isin(indicators_to_use)]

    with output_df:
        if filtered.empty:
            print("⚠️ No data matching the selected filters.")
        else:
            display(filtered.head(100))  # limit for visibility

    with output_stats:
        if not filtered.empty:
            stats = filtered.groupby("Impact Indicator")["value"].agg(['mean', 'std', 'min', 'max', 'median'])
            stats["std/mean"] = (stats["std"] / stats["mean"]).round(4)
            cols = ['mean', 'std', 'std/mean', 'min', 'max', 'median']
            display(stats[cols].round(4))

# Observe changes
for widget in [case_dropdown, scenario_dropdown, family_dropdown, indicator_selector]:
    widget.observe(update_df, names='value')

############## BUILD USER INTERFACE ##############
filter_box = widgets.HBox([case_dropdown, scenario_dropdown, family_dropdown])
display(widgets.Label("🔍 Filter Data"), filter_box)
display(widgets.Label("📌 Select Impact Indicators"))
display(indicator_selector)
# display(output_df)
display(widgets.Label("📊 Summary Statistics"), output_stats)

# Initial display
update_df()


Label(value='🔍 Filter Data')

Label(value='📌 Select Impact Indicators')

ToggleButtons(description='Indicator Set:', options=('Default Indicators', 'All Indicators'), style=ToggleButt…

Label(value='📊 Summary Statistics')

Output()

<div style="border: 2px solid rgba(0, 116, 130, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 116, 130, 0.2); color: black; text-align: center;">
  <h2 style="margin: 0;">Contribution Analysis (CA) data importation</h2>
</div>

**Read and extract the LCIA results data from the Contribution Analysis:**

**The "*Load+Excess-GridDE*" case is treated separately for convenience**:
- Absolute values of the fictional "*Load+Excess+GridDE*" case were computed, separating Load, Excess, and Grid (DE)
- Then, Load and Excess relative shares are calculated as in the "*Load+Excess*" case
- Grid (DE) negative relative share is calculated by normalizing its negative value against thE sum of Load+Excess
- This ensure better comparison against the "*Load+Excess*" case as it allows to show how much the prevented impacts of Grid (DE) represent compared to the "*Load+Excess*" case  

**Optional if you already have the pickle files**

In [ ]:
############## LOAD DATA ##############
# Base directory
base_dir = Path.cwd() / "CA" / "RF"

# Create output directory
pickles_dir = Path.cwd() / "pickles"
pickles_dir.mkdir(exist_ok=True)

for sub in base_dir.iterdir():
    if not sub.is_dir():
        continue

    df_list = []

    for xlsx in sub.glob("*.xlsx"):
        # Metadata lookup
        row = meta_df[meta_df['CA_RF Filename'] == xlsx.name]
        if row.empty:
            raise ValueError(f"No metadata for file: {xlsx.name}")
        md = row.iloc[0].to_dict()

        # Read Excel
        tmp = pd.read_excel(xlsx, sheet_name=0)
        tmp = tmp.iloc[:, 1:]  # Drop first unnamed column
        tmp.columns = ['index'] + list(tmp.columns[1:])

        # Melt to long format
        long = tmp.melt(id_vars='index', var_name='Scenario_Full', value_name='Value')

        # Remove unwanted rows
        long = long[~long['index'].isin(['Score', 'Rest (+)', 'Rest (-)'])]

        # Parse Scenario_Full
        parts = long['Scenario_Full'].str.split('|', expand=True)
        long['Scenario Family'] = parts[0].str.strip()
        long['Scenario'] = parts[1].str.extract(r'(SIM\d{2})')[0]
        long['Country'] = parts[2].str.strip()
        long['Unit'] = parts[3].str.strip()
        long['Label'] = parts[4].str.strip()

        # Rename & enrich
        long = long.rename(columns={'index': 'Contributor'})
        long['H2 storage']      = long['Scenario'].map(h2_map)
        long['Scenario family'] = long['Scenario'].map(fam_map)
        for col in ['Impact Method', 'Impact Category', 'Impact Indicator',
                    'Short Name', 'Unit', 'Robustness']:
            long[col] = md[col]

        df_list.append(long)

    # Concatenate
    result = pd.concat(df_list, ignore_index=True)

    ############## Special case: Load+Excess-GridDE ##############
    if sub.name == "Load+Excess-GridDE":
        # 1) keep original
        result['Original Value'] = result['Value']

        # 2) compute per-(Scenario,Impact Indicator) denom = |LOAD|+|ELEC EXCESS|
        #    using only those two contributors:
        denom = (
            result[result['Contributor'].isin(['LOAD', 'ELEC EXCESS'])]
            .groupby(['Scenario', 'Impact Indicator'])['Original Value']
            .apply(lambda s: s.abs().sum())
            .rename('Denominator')
        )
        # map back to every row
        result = result.join(
            denom, on=['Scenario', 'Impact Indicator']
        )

        # 3) build masks
        is_load   = result['Contributor'] == 'LOAD'
        is_excess = result['Contributor'] == 'ELEC EXCESS'
        is_eco    = result['Contributor'] == 'ecoinvent-3.10-cutoff'

        # 4) apply RSV only where denom != 0
        nonzero = result['Denominator'] != 0

        # LOAD
        mask = is_load & nonzero
        result.loc[mask, 'Value'] = (
            result.loc[mask, 'Original Value'].abs()
            / result.loc[mask, 'Denominator']
        )

        # ELEC EXCESS
        mask = is_excess & nonzero
        result.loc[mask, 'Value'] = (
            result.loc[mask, 'Original Value'].abs()
            / result.loc[mask, 'Denominator']
        )

        # ecoinvent-3.10-cutoff → always negative share
        mask = is_eco & nonzero
        result.loc[mask, 'Value'] = -(
            result.loc[mask, 'Original Value'].abs()
            / result.loc[mask, 'Denominator']
        )

        # 5) cleanup
        result.drop(columns=['Denominator'], inplace=True)

    ############## EXPORT AND DISPLAY ##############
    # Save to pickle
    out_path = pickles_dir / f"{sub.name}_CA_RF.pkl"
    joblib.dump(result, out_path)

    # Assign globally & display
    globals()[sub.name] = result
    print(f"Subfolder '{sub.name}': {len(result)} rows → saved to {out_path}")
    display(result)

<div style="border: 2px solid rgba(0, 116, 130, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 116, 130, 0.2); color: black; text-align: center;">
  <h2 style="margin: 0;">CA data visualization</h2>
</div>

**Display interactive widget for barplots per Reference flow:**

**Only selected EF v3.1 impact methods as default**:
- That can be changed in the "Families & Indicators" menu

**Help for interpretation:**
- Within each family, progression follows decreasing H2 storage capacity in m3 (1000, 500, 100, 50, 30, 10, 1)
- The rows represent the environmental category tested
- Columns represent the impact boundary

In [4]:
# Set bold font from matplotlib properties
bold_font = FontProperties(weight='bold')

############## LOAD AND LABEL CONTRIBUTION DATA ##############
# Load and label
pickle_files = [
    "pickles/Load_CA_RF.pkl",
    "pickles/Load+Excess_CA_RF.pkl",
    "pickles/Load+Excess-GridDE_CA_RF.pkl"
]
df_list = []
for fpath in pickle_files:
    tmp = joblib.load(fpath)
    key = os.path.splitext(os.path.basename(fpath))[0].replace('_CA_RF', '')
    tmp['Source'] = key

    # Extract scenario number and assign families based on the extracted number
    tmp["Scenario Number"] = tmp["Scenario"].str.extract(r'(\d+)').astype(int)

    # Corrected np.select usage to assign families
    conditions = [
        tmp["Scenario Number"] <= 7,
        (tmp["Scenario Number"] >= 8) & (tmp["Scenario Number"] <= 14),
        (tmp["Scenario Number"] >= 15) & (tmp["Scenario Number"] <= 21),
        tmp["Scenario Number"] >= 22
    ]
    families = ["PV + WT + BAT", "WT + BAT", "PV + BAT", "PV"]
    tmp["Family"] = np.select(conditions, families, default="")

    df_list.append(tmp)
df = pd.concat(df_list, ignore_index=True)

# Default selections
default_indicators = [
    "Global Warming Potential (Climate Change)",
    "Ecotoxicity (Freshwater)",
    "Human Toxicity (Carcinogenic)",
    "Abiotic Depletion Potential (Ultimate Reserves)",
    "User Deprivation Potential (Water Use)"
]
all_indicators = sorted(df["Impact Indicator"].unique())
selected_defaults = [ind for ind in default_indicators if ind in all_indicators]
all_families = sorted(df["Scenario family"].unique())

# Column title formatting
def format_title(key):
    t = re.sub(r"\s*[+]\s*", " + ", key)
    t = re.sub(r"\s*[-]\s*", " - ", t)
    t = t.replace('GridDE', 'Grid (DE)')
    return t.strip()
cols = [os.path.splitext(os.path.basename(f))[0].replace('_CA_RF', '') for f in pickle_files]
formatted_titles = [format_title(c) for c in cols]

# Define contributors and shared color palette (colorblind-safe)
contributors = ["BAT", "GSHP", "H2S", "ICE-CHP", "PV", "WT", "ecoinvent-3.10-cutoff", "LOAD", "ELEC EXCESS"]

color_palette = sns.color_palette("colorblind", n_colors=len(contributors))
contrib_color_dict = dict(zip(contributors, color_palette))

# Mapping for legend labels
legend_labels = {"ecoinvent-3.10-cutoff": "GRID"}

############## WIDGETS ##############
family_sel = widgets.SelectMultiple(
    options=all_families,
    value=tuple(all_families),
    description="Scenario Families",
    style={"description_width":"initial"},
    layout=widgets.Layout(width="350px", height="120px")
)
ind_sel = widgets.SelectMultiple(
    options=all_indicators,
    value=tuple(selected_defaults),
    description="Impact Indicators",
    style={"description_width":"initial"},
    layout=widgets.Layout(width="350px", height="180px")
)
savefig = widgets.ToggleButtons(
    options=["unsaved", "saved"],
    value="unsaved",
    description="Save Figure",
    style={"description_width": "initial"}
)

############## PLOT ##############
def update_plot(fams, inds, savefig):
    if not fams or not inds:
        print("Select at least one family and one indicator.")
        return
    sub = df[df["Scenario family"].isin(fams) & df["Impact Indicator"].isin(inds)].copy()

    # Ensure that sub is ordered by default_indicators (this ensures custom order)
    sub["Impact Indicator"] = pd.Categorical(sub["Impact Indicator"], categories=default_indicators, ordered=True)
    sub = sub.sort_values(by="Impact Indicator")

    g = sns.FacetGrid(
        sub,
        row="Impact Indicator",
        row_order=default_indicators,  # Explicitly set the row order to follow default_indicators
        col="Source",
        col_order=cols,
        height=3,
        aspect=1.4,
        sharey=False,
        sharex=True
    )

    def plot_stacked_bar(data, ax):
        pivot_data = data.pivot_table(index='Scenario Number', columns='Contributor', values='Value', aggfunc='sum', fill_value=0)

        # Keep only contributors we defined
        pivot_data = pivot_data[[c for c in contributors if c in pivot_data.columns]]

        # Stack plot
        pivot_data.plot(
            kind='bar',
            stacked=True,
            ax=ax,
            color=[contrib_color_dict[c] for c in pivot_data.columns],
            width=0.7,
            legend=False,
            alpha=1,
        )

        ax.set_ylabel("Relative Share")
        ax.set_xlabel("Scenario Number")
        title = data['Impact Indicator'].iloc[0]
        if "(" in title:
            before_paren, after_paren = title.split("(", 1)
            formatted_title = f"{before_paren.strip()}\n({after_paren.strip()}"
        else:
            formatted_title = title
        ax.set_title(formatted_title, pad=5, fontsize=10)

        # ------ Remove all x-axis ticks and labels ------
        ax.set_xticks([])  # Removes x-ticks
        ax.set_xticklabels([])  # Removes x-tick labels
        ax.tick_params(axis='x', which='both', length=0)  # Removes both major and minor ticks
        ax.minorticks_off()  # Disable minor ticks completely
        # ---------------------------------------------------

    for ax, (_, data) in zip(g.axes.flat, sub.groupby(['Impact Indicator', 'Source'], observed=False)):
        plot_stacked_bar(data, ax)

    # Titles and labels
    for i, ind in enumerate(inds):
        for j, src in enumerate(cols):
            ax = g.axes[i, j]
            if i == 0:
                ax.text(0.5, 1.25, formatted_titles[j], transform=ax.transAxes,
                        ha='center', va='bottom', fontweight='bold')

    # Create a single shared legend manually
    from matplotlib.patches import Patch
    handles = []
    for contrib in contributors:
        label = legend_labels.get(contrib, contrib)
        handles.append(Patch(facecolor=contrib_color_dict[contrib], label=label))

    fig = plt.gcf()
    fig.legend(
        handles=handles,
        title="Contributor",
        title_fontproperties=bold_font,
        loc='lower center',
        bbox_to_anchor=(0.5, -0.05),  # Centered below the plot
        ncol=3,  # 3 columns
        frameon=True,
        borderaxespad=0.7,     # Space between the plot and the legend
        labelspacing=0.7,      # Space between rows (legend entries)
        columnspacing=2,     # Space between columns
        handletextpad=1,     # Space between handle and label
        borderpad=0.6          # Padding inside the legend border (top/bottom)
    )

    # -------------------------------
    # Shared X Axis Adjustments
    # -------------------------------
    family_midpoints = [np.mean([1, 7]), np.mean([8, 14]), np.mean([15, 21]), np.mean([22, 28])]
    family_labels = ["PV + WT + BAT", "WT + BAT", "PV + BAT", "PV"]
    for ax in g.axes.flat:
        ax.set_xticks(family_midpoints)
        ax.set_xticklabels(family_labels, rotation=45, ha='right')  # Set labels at 45° rotation and adjust horizontal alignment
        ax.set_xlabel("Scenario Family", fontweight='bold')  # Adjust the space between labels and xlabel

    plt.tight_layout(rect=[0, 0.03, 1, 1])  # Leaves extra space at bottom

    # Save figure if requested
    if savefig == "saved":
        fig = plt.gcf()
        fig.savefig("LCIA_CA_RF.png", dpi=600, bbox_inches="tight")
        fig.savefig("LCIA_CA_RF.pdf", bbox_inches="tight")
        print("Figure saved as 'LCIA_CA_RF.png' and 'LCIA_CA_RF.pdf'.")

    plt.show()

############## BUILD USER INTERFACE ##############
filter_panel = widgets.VBox([
    widgets.HTML("<b>Filter Options</b>"),
    family_sel,
    ind_sel
])
save_panel = widgets.VBox([
    widgets.HTML("<b>Save Figure</b>"),
    savefig
])
accordion = widgets.Accordion(children=[filter_panel, save_panel])
accordion.set_title(0, "Select Families & Indicators")
accordion.set_title(1, "Save Figure")

display(accordion, widgets.interactive_output(
    update_plot,
    {"fams": family_sel, "inds": ind_sel, "savefig": savefig}
))

Accordion(children=(VBox(children=(HTML(value='<b>Filter Options</b>'), SelectMultiple(description='Scenario F…

Output()

<div style="border: 2px solid rgba(0, 116, 130, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 116, 130, 0.2); color: black; text-align: center;">
  <h2 style="margin: 0;">CA data statistics</h2>
</div>

**Display interactive widget for exploring data of contribution to impacts per Reference flow.**

**Filtering options:**
- Case
- Scenario
- Scenario family
- Contributor

In [5]:
############## LOAD AND LABEL THE DATA ##############
pickle_files = [
    "pickles/Load_CA_RF.pkl",
    "pickles/Load+Excess_CA_RF.pkl",
    "pickles/Load+Excess-GridDE_CA_RF.pkl"
]

def format_title(key):
    t = re.sub(r"\s*[+]\s*", " + ", key)
    t = re.sub(r"\s*[-]\s*", " - ", t)
    t = t.replace('GridDE', 'Grid (DE)')
    return t.strip()

df_list = []
case_keys = []

for fpath in pickle_files:
    tmp = joblib.load(fpath)
    key = os.path.splitext(os.path.basename(fpath))[0].replace('_CA_RF', '')
    tmp['Source'] = key
    tmp["Scenario Number"] = tmp["Scenario"].str.extract(r'(\d+)').astype(int)
    conditions = [
        tmp["Scenario Number"] <= 7,
        (tmp["Scenario Number"] >= 8) & (tmp["Scenario Number"] <= 14),
        (tmp["Scenario Number"] >= 15) & (tmp["Scenario Number"] <= 21),
        tmp["Scenario Number"] >= 22
    ]
    families = ["PV + WT + BAT", "WT + BAT", "PV + BAT", "PV"]
    tmp["Family"] = np.select(conditions, families, default="")
    df_list.append(tmp)
    case_keys.append(key)

df = pd.concat(df_list, ignore_index=True)

############## OPTIONS FOR WIDGETS ##############
formatted_titles = [format_title(c) for c in case_keys]
case_map = dict(zip(formatted_titles, case_keys))

# Contributors and mapping
contributors_raw = ["BAT", "GSHP", "H2S", "ICE-CHP", "PV", "WT", "ecoinvent-3.10-cutoff", "LOAD", "ELEC EXCESS"]
legend_labels = {"ecoinvent-3.10-cutoff": "GRID"}
reverse_legend_labels = {v: k for k, v in legend_labels.items()}
contributor_display_names = [legend_labels.get(c, c) for c in contributors_raw]

# Indicators
default_indicators = [
    "Global Warming Potential (Climate Change)",
    "Ecotoxicity (Freshwater)",
    "Human Toxicity (Carcinogenic)",
    "Abiotic Depletion Potential (Ultimate Reserves)",
    "User Deprivation Potential (Water Use)"
]
all_indicators = sorted(df["Impact Indicator"].unique())

############## WIDGETS ##############
case_dropdown = widgets.SelectMultiple(
    options=formatted_titles,
    description="Case(s):",
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)

scenario_dropdown = widgets.SelectMultiple(
    options=sorted(df["Scenario"].unique()),
    description="Scenario(s):",
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)

family_dropdown = widgets.SelectMultiple(
    options=sorted(df["Family"].dropna().unique()),
    description="Family(ies):",
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)

contributor_dropdown = widgets.SelectMultiple(
    options=contributor_display_names,
    description="Contributor(s):",
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)

indicator_selector = widgets.ToggleButtons(
    options=["Default Indicators", "All Indicators"],
    value="Default Indicators",
    description="Indicator Set:",
    style={'description_width': 'initial'}
)

# Output areas ==========
output_df = widgets.Output()
output_stats = widgets.Output()

############## FILTER AND DISPLAY DATAFRAME ##############
def update_df(change=None):
    output_df.clear_output()
    output_stats.clear_output()

    selected_cases = [case_map[c] for c in case_dropdown.value]
    filtered = df[df["Source"].isin(selected_cases)]

    if scenario_dropdown.value:
        filtered = filtered[filtered["Scenario"].isin(scenario_dropdown.value)]

    if family_dropdown.value:
        filtered = filtered[filtered["Family"].isin(family_dropdown.value)]

    if contributor_dropdown.value:
        selected_contributors = [
            reverse_legend_labels.get(label, label) for label in contributor_dropdown.value
        ]
        filtered = filtered[filtered["Contributor"].isin(selected_contributors)]

    # Indicator filtering
    if indicator_selector.value == "Default Indicators":
        indicators_to_use = default_indicators
    else:
        indicators_to_use = all_indicators

    filtered = filtered[filtered["Impact Indicator"].isin(indicators_to_use)]

    # Replace contributor label for display
    filtered = filtered.copy()
    filtered["Contributor"] = filtered["Contributor"].replace(legend_labels)

    with output_df:
        if filtered.empty:
            print("⚠️ No data matching the selected filters.")
        else:
            display(filtered.head(100))  # limit for visibility

    with output_stats:
        if not filtered.empty:
            stats = filtered.groupby("Impact Indicator")["Value"].agg(['mean', 'std', 'min', 'max', 'median'])
            stats["std/mean"] = (stats["std"] / stats["mean"]).round(4)
            # Reorder columns to put "std/mean" after "std"
            cols = ['mean', 'std', 'std/mean', 'min', 'max', 'median']
            display(stats[cols].round(4))

# Observe changes
for widget in [case_dropdown, scenario_dropdown, family_dropdown, contributor_dropdown, indicator_selector]:
    widget.observe(update_df, names='value')

############## BUILD USER INTERFACE ##############
filter_box = widgets.HBox([case_dropdown, scenario_dropdown, family_dropdown, contributor_dropdown])
display(widgets.Label("🔍 Filter Data"), filter_box)
display(widgets.Label("📌 Select Impact Indicators"))
display(indicator_selector)
#display(output_df)
display(widgets.Label("📊 Summary Statistics"), output_stats)

# Initial display
update_df()

Label(value='🔍 Filter Data')

Label(value='📌 Select Impact Indicators')

ToggleButtons(description='Indicator Set:', options=('Default Indicators', 'All Indicators'), style=ToggleButt…

Label(value='📊 Summary Statistics')

Output()

<div style="border: 2px solid #9f8189; padding: 10px; border-radius: 5px; background-color: #ffcad4; color: black; text-align: center;">
  <h2 style="margin: 0;">Global Sensitivity Analysis (GSA) data importation</h2>
</div>

**Read and extract the LCIA results data from the Global Sensitivity Analysis:**

**Optional if you already have the pickle files**

In [ ]:
############## LOAD AND LABEL THE GLOBAL SENSITIVITY DATA ##############
# Base directory containing subfolders
base_dir = Path.cwd() / "GSA"

# Create a directory for pickles
pickles_dir = Path.cwd() / "pickles"
pickles_dir.mkdir(exist_ok=True)

# Loop over each top‐level folder (Load, Load+Excess, Load+Excess-GridDE)
for sub in base_dir.iterdir():
    if not sub.is_dir():
        continue

    df_list = []

    # Within each, loop over scenario subfolders sorted SIM01 → SIM28
    for scen_dir in sorted(sub.iterdir(), key=lambda p: int(p.name[3:])):
        if not scen_dir.is_dir():
            continue

        scenario_name = scen_dir.name  # e.g. "SIM01"

        # Process all .xlsx files in this scenario folder
        for xlsx in scen_dir.glob("*.xlsx"):
            # metadata lookup: exact match on 'GSA Filename'
            row = meta_df[meta_df['GSA Filename'] == xlsx.name]
            if row.empty:
                raise ValueError(f"No metadata for file: {xlsx.name}")
            md = row.iloc[0].to_dict()

            # Read Excel and drop the first (unnamed) column
            tmp = pd.read_excel(xlsx, sheet_name=0)
            tmp = tmp.iloc[:, 1:]  # drop the auto-generated index column

            # Now tmp still has all original columns:
            # ['GSA name', 'delta', 'delta_conf', 'S1', 'S1_conf', 'names', …, 'formula']

            # Add Scenario & maps
            tmp['Scenario']        = scenario_name
            tmp['H2 storage']      = scenario_name.map(h2_map) if hasattr(scenario_name, 'map') else h2_map.get(scenario_name)
            tmp['Scenario family'] = scenario_name.map(fam_map) if hasattr(scenario_name, 'map') else fam_map.get(scenario_name)

            # Add metadata columns
            for col in [
                'Impact Method',
                'Impact Category',
                'Impact Indicator',
                'Short Name',
                'Unit',
                'Robustness'
            ]:
                tmp[col] = md[col]

            df_list.append(tmp)

    # Concatenate all results for this top-level folder
    result = pd.concat(df_list, ignore_index=True)

    ############## EXPORT AND DISPLAY ##############
    # Save as pickle with _GSA suffix
    out_path = pickles_dir / f"{sub.name}_GSA.pkl"
    joblib.dump(result, out_path)

    # Optionally assign to a variable named after the folder
    globals()[sub.name] = result

    print(f"Folder '{sub.name}': {len(result)} rows → saved to {out_path}")
    display(result)

<div style="border: 2px solid #9f8189; padding: 10px; border-radius: 5px; background-color: #ffcad4; color: black; text-align: center;">
  <h2 style="margin: 0;">GSA data visualization</h2>
</div>

<div style="border: 2px solid #9f8189; padding: 10px; border-radius: 5px; background-color: #ffe4e9; color: black; text-align: center;">
  <h2 style="margin: 0;">Load the data</h2>
</div>

In [6]:
pickles_dir = Path.cwd() / "pickles"
cases = {}
for pkl in pickles_dir.glob("*_GSA.pkl"):
    case_name = pkl.stem.replace("_GSA", "")
    cases[case_name] = joblib.load(pkl)

# Quick check
for name, df in cases.items():
    print(f"{name}: {len(df)} rows, columns = {list(df.columns)}")

Load: 17805 rows, columns = ['GSA name', 'delta', 'delta_conf', 'S1', 'S1_conf', 'names', 'activity', 'amount', 'classifications', 'comment', 'flow', 'from location', 'from name', 'index', 'input', 'loc', 'maximum', 'minimum', 'name', 'negative', 'output', 'pedigree', 'production volume', 'properties', 'scale', 'scale without pedigree', 'shape', 'to location', 'to name', 'type', 'uncertainty type', 'unit', 'formula', 'Scenario', 'H2 storage', 'Scenario family', 'Impact Method', 'Impact Category', 'Impact Indicator', 'Short Name', 'Unit', 'Robustness']
Load+Excess: 17041 rows, columns = ['GSA name', 'delta', 'delta_conf', 'S1', 'S1_conf', 'names', 'activity', 'amount', 'classifications', 'comment', 'flow', 'from location', 'from name', 'index', 'input', 'loc', 'maximum', 'minimum', 'name', 'negative', 'output', 'pedigree', 'production volume', 'properties', 'scale', 'scale without pedigree', 'shape', 'to location', 'to name', 'type', 'uncertainty type', 'unit', 'formula', 'Scenario', 'H

<div style="border: 2px solid #9f8189; padding: 10px; border-radius: 5px; background-color: #ffe4e9; color: black; text-align: center;">
  <h2 style="margin: 0;">Rank correlation & sets similarity</h2>
</div>

**Display interactive widget for heatmaps related to scenario to scenario comparison of GSA results:**

**2 available indicators**:
- Jaccard similarity index (by default) / sets similarity (crude but useful indicator)
- Kendall τ coefficient / ranking correlation (more detailed but unhelpful if too much processes have similar influence)

**Help for interpretation:**
- Within each family, progression follows decreasing H2 storage capacity in m3 (1000, 500, 100, 50, 30, 10, 1)
- The rows represent the environmental category tested
- Columns represent the impact boundary

In [7]:
############## WIDGETS ##############
top_n_slider = widgets.IntSlider(
    value=10, min=5, max=300, step=5,
    description='Top N:', continuous_update=False,
    style={'description_width':'initial'}
)
metric_sel = widgets.ToggleButtons(
    options=["Jaccard", "Kendall"],
    value="Jaccard", description="Metric:",
    style={"description_width":"initial"}
)
savefig = widgets.ToggleButtons(
    options=["unsaved","saved"], value="unsaved",
    description="Save Figure",
    style={"description_width":"initial"}
)

# Build the accordion UI
metric_panel = widgets.VBox([widgets.HTML("<b>Select Metric</b>"), metric_sel, top_n_slider])
save_panel   = widgets.VBox([widgets.HTML("<b>Save Figure</b>"), savefig])
accordion    = widgets.Accordion([metric_panel, save_panel])
accordion.set_title(0, "Metric & Top-N")
accordion.set_title(1, "Save Figure")

display(accordion)

############## UPDATE FUNCTION ##############
def update_stability(metric, top_n, savefig):
    cases_to_plot = ['Load','Load+Excess','Load+Excess-GridDE']
    # format case titles
    formatted_titles = []
    for c in cases_to_plot:
        t = re.sub(r"\s*[+]\s*"," + ",c)
        t = re.sub(r"\s*[-]\s*"," - ",t)
        t = t.replace('GridDE','Grid (DE)')
        formatted_titles.append(t)
    indicators = [
        "Global Warming Potential (Climate Change)",
        "Ecotoxicity (Freshwater)",
        "Human Toxicity (Carcinogenic)",
        "Abiotic Depletion Potential (Ultimate Reserves)",
        "User Deprivation Potential (Water Use)"
    ]
    scenarios = [f"SIM{str(i).zfill(2)}" for i in range(1,29)]
    family_midpoints = [np.mean([0,6]),np.mean([7,13]),np.mean([14,20]),np.mean([21,27])]
    family_labels = ["PV + WT + BAT","WT + BAT","PV + BAT","PV"]

    def compute_matrix(df_sub):
        # prepare structure
        mat = pd.DataFrame(index=scenarios, columns=scenarios, dtype=float)
        if metric=="Kendall":
            pv = df_sub.pivot(index='GSA name', columns='Scenario', values='delta')
        # precompute top_N sets
        top_sets = {
            sc: set(df_sub[df_sub['Scenario']==sc]
                    .nlargest(top_n,'delta')['GSA name'])
            for sc in scenarios
        }
        for a,b in combinations(scenarios,2):
            if metric=="Jaccard":
                s1,s2=top_sets[a],top_sets[b]
                val = len(s1&s2)/len(s1|s2) if (s1|s2) else np.nan
            else:
                val,_ = kendalltau(pv[a],pv[b],nan_policy='omit')
            mat.loc[a,b] = mat.loc[b,a] = val
        fill = 1.0
        np.fill_diagonal(mat.values, fill)
        return mat

    ############## PLOT FUNCTION ##############
    sns.set(style="white")
    n_ind,n_cases = len(indicators),len(cases_to_plot)
    fig,axes = plt.subplots(n_ind,n_cases,
                            figsize=(4*n_cases,3*n_ind),
                            constrained_layout=True)
    mappable=None

    for i,ind in enumerate(indicators):
        fmt_ind = f"{ind.split('(',1)[0].strip()}\n({ind.split('(',1)[1]}" if '(' in ind else ind
        for j,case in enumerate(cases_to_plot):
            ax = axes[i,j]
            df_ind = cases[case][cases[case]['Impact Indicator']==ind]
            mat = compute_matrix(df_ind)
            cmap = 'mako' if metric=="Jaccard" else 'vlag'
            vmin,vmax = (0,1) if metric=="Jaccard" else (-1,1)
            hm = sns.heatmap(mat,ax=ax,cmap=cmap,vmin=vmin,vmax=vmax,
                             cbar=False,square=True,
                             linewidths=0.3,linecolor='gray')
            if mappable is None:
                mappable = hm.get_children()[0]

            # case title on row 0
            if i==0:
                ax.text(0.5,1.20,formatted_titles[j],
                        transform=ax.transAxes,
                        ha='center',va='bottom',
                        fontweight='bold',fontsize=10)
            # indicator title
            ax.set_title(fmt_ind,pad=5,fontsize=10)

            # X only bottom row
            if i==n_ind-1:
                ax.set_xticks(family_midpoints)
                ax.set_xticklabels(family_labels,rotation=45,ha='right')
                ax.set_xlabel("Scenario Family",fontweight='bold')
            else:
                ax.set_xticks([]); ax.set_xlabel('')

            # Y axis on every subplot
            ax.set_yticks(family_midpoints)
            ax.set_yticklabels(family_labels, rotation=0, va='center')
            if j==0:
                ax.set_ylabel("Scenario Family", fontweight='bold')
            else:
                ax.set_ylabel('')

    # horizontal colorbar
    bold_font=FontProperties(weight='bold')
    facet_w=1.0/n_cases
    left=(1-facet_w)/2; width=facet_w*0.8
    cax=fig.add_axes([left,-0.06,width,0.02])
    ticks = np.linspace(vmin,vmax,6)
    cb=fig.colorbar(mappable,cax=cax,orientation='horizontal',ticks=ticks)
    label = f"{metric}" + (" similarity" if metric=="Jaccard" else " (τ)")
    cb.set_label(label,fontsize=12,fontproperties=bold_font,labelpad=5)
    cb.ax.tick_params(labelsize=10)

    # save
    if savefig=="saved":
        fig.savefig(f"GSA_{metric}_Top{top_n}.png",dpi=600,bbox_inches="tight")
        fig.savefig(f"GSA_{metric}_Top{top_n}.pdf",bbox_inches="tight")
        print(f"Saved GSA_{metric}_Top{top_n}.png/pdf")

    plt.show()

############## OUTPUTS ##############
out = widgets.interactive_output(
    update_stability,
    {"metric":metric_sel,"top_n":top_n_slider,"savefig":savefig}
)
display(out)

Accordion(children=(VBox(children=(HTML(value='<b>Select Metric</b>'), ToggleButtons(description='Metric:', op…

Output()

<div style="border: 2px solid #9f8189; padding: 10px; border-radius: 5px; background-color: #ffe4e9; color: black; text-align: center;">
  <h2 style="margin: 0;">Scenario global δ statistics</h2>
</div>

**Display typical statistics: mean, median, std, min, max, variance, for δ & δ_conf**

In [8]:
# Pick a case
case = 'Load'

# Compute aggregated statistics
agg = (cases[case]
       .groupby(['Scenario family','Impact Indicator'])
       .agg(delta_mean=('delta','mean'),
            delta_median=('delta','median'),
            delta_std=('delta','std'),
            delta_min=('delta','min'),
            delta_max=('delta','max'),
            delta_var=('delta','var'),
            conf_mean=('delta_conf','mean'),
            conf_median=('delta_conf','median'),
            conf_var=('delta_conf','var'))
       .reset_index())

# Display
agg.head(20)

,Scenario family,Impact Indicator,delta_mean,delta_median,delta_std,delta_min,delta_max,delta_var,conf_mean,conf_median,conf_var
0,PV,Abiotic Depletion Potential (Ultimate Reserves),0.069965,0.066793,0.031553,0.015755,0.370782,0.000996,0.016625,0.016599,0.000003
1,PV,Ecotoxicity (Freshwater),0.071915,0.070124,0.017684,0.023206,0.136081,0.000313,0.016203,0.016134,0.000003
2,PV,Global Warming Potential (Climate Change),0.069560,0.064936,0.032754,0.021569,0.411310,0.001073,0.016001,0.016133,0.000006
3,PV,Human Toxicity (Carcinogenic),0.069301,0.067877,0.020444,0.021559,0.226617,0.000418,0.016052,0.016064,0.000004
4,PV,User Deprivation Potential (Water Use),0.074683,0.066552,0.066079,0.014011,0.701303,0.004366,0.016488,0.016508,0.000003
5,PV + BAT,Abiotic Depletion Potential (Ultimate Reserves),0.070167,0.067430,0.024182,0.021568,0.273334,0.000585,0.016719,0.016631,0.000004
6,PV + BAT,Ecotoxicity (Freshwater),0.071404,0.070173,0.018116,0.022200,0.157126,0.000328,0.016299,0.016168,0.000003
7,PV + BAT,Global Warming Potential (Climate Change),0.068537,0.066142,0.028322,0.017736,0.446463,0.000802,0.015931,0.016008,0.000006
8,PV + BAT,Human Toxicity (Carcinogenic),0.068166,0.066762,0.019163,0.017547,0.232561,0.000367,0.015997,0.015974,0.000004
9,PV + BAT,User Deprivation Potential (Water Use),0.073312,0.067352,0.053051,0.017838,0.582169,0.002814,0.016531,0.016506,0.000003


<div style="border: 2px solid #9f8189; padding: 10px; border-radius: 5px; background-color: #ffe4e9; color: black; text-align: center;">
  <h2 style="margin: 0;">Spread and maximums of δ values</h2>
</div>

**“Sensitivity signatures”: process‐spread = max(δ)–min(δ) over all Scenarios**


In [9]:
bold_font = FontProperties(weight="bold")

############## LABEL THE DATA ##############
# Clean Case Titles
def format_title(text):
    t = re.sub(r"\s*[+]\s*", " + ", text)
    t = re.sub(r"\s*[-]\s*", " - ", t)
    t = t.replace('GridDE', 'Grid (DE)')
    return t.strip()

# Sample case and indicator lists
case_list = ['Load', 'Load+Excess', 'Load+Excess-GridDE']
formatted_titles = [format_title(c) for c in case_list]
case_title_dict = dict(zip(case_list, formatted_titles))

indicator_list = [
    "Global Warming Potential (Climate Change)",
    "Ecotoxicity (Freshwater)",
    "Human Toxicity (Carcinogenic)",
    "Abiotic Depletion Potential (Ultimate Reserves)",
    "User Deprivation Potential (Water Use)"
]

# Clean function
def clean_gsa_name(name):
    prefix = ""
    if name.startswith("T:"):
        from_index = name.find("FROM")
        if from_index != -1:
            prefix = name[:from_index].strip()
            # Replace "FROM ... TO" with "//"
            name = re.sub(r"FROM.*?TO", "//", name[from_index:], flags=re.IGNORECASE).strip()
        else:
            prefix = "T:"
            name = name[2:].strip()
    if "TO" in name:
        name = name.split("TO", 1)[1].strip()
    name = re.sub(r"\s*\(.*?\)", "", name)
    name = re.sub(r"\s*\[.*?\]\s*$", "", name)
    return (prefix + " " + name).strip() if prefix else name.strip()

############## BUILD DATAFRAME ############## 
# Assuming 'cases' dict already defined)
all_data_points = []
for case in case_list:
    for indicator in indicator_list:
        sub = cases[case][cases[case]['Impact Indicator'] == indicator]
        if sub.empty:
            continue
        sub = sub.copy()
        sub['case'] = case
        sub['indicator'] = indicator
        sub['GSA name cleaned'] = sub['GSA name'].apply(clean_gsa_name)
        all_data_points.append(sub[['case', 'indicator', 'GSA name', 'GSA name cleaned', 'delta', 'Scenario family']])

all_data_df = pd.concat(all_data_points, ignore_index=True)

# Compute spread
spread_df = (
    all_data_df
    .groupby(['case', 'indicator', 'GSA name', 'GSA name cleaned'])
    .agg(delta_max=('delta', 'max'), delta_min=('delta', 'min'))
    .reset_index()
)
spread_df['spread'] = spread_df['delta_max'] - spread_df['delta_min']

# Prepare color palette
scenario_families = ['PV + WT + BAT', 'WT + BAT', 'PV + BAT', 'PV']
palette = sns.color_palette("colorblind", n_colors=len(scenario_families))
scenario_family_colors = dict(zip(scenario_families, palette))

############## WIDGETS ##############
view_selector = widgets.Dropdown(
    options=[
        ('Order by spread (overall top N)', 'default'),
        ('Top N per case', 'by_case'),
        ('Top N per indicator', 'by_indicator')
    ],
    description='View mode:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

metric_toggle = widgets.ToggleButtons(
    options=[("δ spread", "spread"), ("δ max", "delta_max")],
    value="spread",
    description="Metric:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

top_n_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=50,
    step=5,
    description='Top N:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display_mode_toggle = widgets.ToggleButtons(
    options=[
        ("By case & By indicator", "both"),
        ("By case only", "case_only"),
        ("By indicator only", "indicator_only")
    ],
    value="both",
    description="Display mode:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

savefig = widgets.ToggleButtons(
    options=["unsaved", "saved"],
    value="unsaved",
    description="Save Figure",
    style={"description_width": "initial"}
)

############## BUILD USER INTERFACE ##############
view_selector_panel = widgets.VBox([
    widgets.HTML("<b>Select Ordering, Metric & Threshold</b>"),
    view_selector,
    metric_toggle,
    top_n_slider
])
save_panel = widgets.VBox([widgets.HTML("<b>Save Figure</b>"), savefig])
display_mode_panel = widgets.VBox([widgets.HTML("<b>Display Mode</b>"), display_mode_toggle])

accordion = widgets.Accordion([view_selector_panel, save_panel, display_mode_panel])
accordion.set_title(0, "Ordering, Metric & Top-N")
accordion.set_title(1, "Save Figure")
accordion.set_title(2, "Display Mode")

display(accordion)

output = widgets.Output()

############## PLOT AND UPDATE ##############
def update_all(view_mode, metric, top_n, save_state, display_mode):
    """Redraw the figure and table every time a control changes."""
    with output:
        clear_output(wait=True)

        # Choose the column used for ranking
        rank_col = metric                     # 'spread' or 'delta_max'

        # Select the Top-N rows
        if view_mode == 'default':
            selected = (spread_df
                        .sort_values(rank_col, ascending=False)
                        .head(top_n))
        elif view_mode == 'by_case':
            selected = (spread_df
                        .sort_values(['case', rank_col],
                                      ascending=[True, False])
                        .groupby('case')
                        .head(top_n))
        elif view_mode == 'by_indicator':
            selected = (spread_df
                        .sort_values(['indicator', rank_col],
                                     ascending=[True, False])
                        .groupby('indicator')
                        .head(top_n))
        else:  # fall-back
            selected = spread_df

        ############## WHICH COLUMN / ROW TO PLOT? ##############
        show_case      = display_mode in ("both", "case_only")
        show_indicator = display_mode in ("both", "indicator_only")
        n_rows = sum([show_case, show_indicator])
        n_cols = max(len(case_list) if show_case else 0,
                     len(indicator_list) if show_indicator else 0)

        fig_width  = 4.25 * n_cols
        fig_height = 7   * n_rows
        fig, axes  = plt.subplots(n_rows, n_cols,
                                  figsize=(fig_width, fig_height),
                                  sharey='row', sharex=True,
                                  layout='constrained')

        # When only one row, make axes 2-d for uniform indexing
        if n_rows == 1:
            axes = axes.reshape(1, -1)

        legend_handles = legend_labels = None
        row_idx = 0

        ############## ROW BY CASE ##############
        if show_case:
            for i, case in enumerate(case_list):
                ax = axes[row_idx, i]
                subset = selected[selected['case'] == case]
                gsa_order = (subset
                             .sort_values(rank_col, ascending=False)
                             ['GSA name cleaned']
                             .unique())

                plot_df = all_data_df[(all_data_df['case'] == case) &
                                      (all_data_df['GSA name cleaned']
                                       .isin(gsa_order))].copy()
                if not plot_df.empty:
                    plot_df['GSA name cleaned'] = pd.Categorical(
                        plot_df['GSA name cleaned'],
                        categories=gsa_order, ordered=True)

                    sns.stripplot(data=plot_df,
                                  x='delta',
                                  y='GSA name cleaned',
                                  hue='Scenario family',
                                  palette=scenario_family_colors,
                                  ax=ax, orient='h',
                                  jitter=0.2, size=4, dodge=True,
                                  alpha=0.8, edgecolor="black",
                                  linewidth=0.2)

                    if legend_handles is None:
                        legend_handles, legend_labels = ax.get_legend_handles_labels()

                ax.set_title(case_title_dict[case], fontproperties=bold_font)
                ax.set_xlabel('δ value', fontsize=12, fontproperties=bold_font)
                ax.set_ylabel('')
                ax.grid(True, axis='x', linestyle='-', alpha=0.3)
                ax.grid(True, axis='y', linestyle='-', alpha=0.3)
                if ax.legend_:
                    ax.legend_.remove()

            # Hide empty axes
            for j in range(len(case_list), n_cols):
                axes[row_idx, j].axis('off')

            row_idx += 1

        ############## ROW BY INDICATOR ##############
        if show_indicator:
            for i, indicator in enumerate(indicator_list):
                ax = axes[row_idx, i]
                title = re.sub(r'\s*\((.*?)\)', r'\n(\1)', indicator)
                subset = selected[selected['indicator'] == indicator]
                gsa_order = (subset
                             .sort_values(rank_col, ascending=False)
                             ['GSA name cleaned']
                             .unique())

                plot_df = all_data_df[(all_data_df['indicator'] == indicator) &
                                      (all_data_df['GSA name cleaned']
                                       .isin(gsa_order))].copy()
                if not plot_df.empty:
                    plot_df['GSA name cleaned'] = pd.Categorical(
                        plot_df['GSA name cleaned'],
                        categories=gsa_order, ordered=True)

                    sns.stripplot(data=plot_df,
                                  x='delta',
                                  y='GSA name cleaned',
                                  hue='Scenario family',
                                  palette=scenario_family_colors,
                                  ax=ax, orient='h',
                                  jitter=0.2, size=4, dodge=True,
                                  alpha=0.8, edgecolor="black",
                                  linewidth=0.2)

                    if legend_handles is None:
                        legend_handles, legend_labels = ax.get_legend_handles_labels()

                ax.set_title(title, fontproperties=bold_font)
                ax.set_xlabel('δ value', fontsize=12, fontproperties=bold_font)
                ax.set_ylabel('')
                ax.grid(True, axis='x', linestyle='-', alpha=0.3)
                ax.grid(True, axis='y', linestyle='-', alpha=0.3)
                if ax.legend_:
                    ax.legend_.remove()

            # Hide empty axes
            for j in range(len(indicator_list), n_cols):
                axes[row_idx, j].axis('off')

        ############## SCENARIO FAMILY LEGEND ##############
        legend_settings = {
            "both"          : {"loc": "upper right", "bbox_to_anchor": (0.83, 0.83)},
            "case_only"     : {"loc": "upper right", "bbox_to_anchor": (1.15, 0.65)},
            "indicator_only": {"loc": "upper right", "bbox_to_anchor": (1.09, 0.65)}
        }
        if legend_handles:
            fig.legend(legend_handles, legend_labels,
                       title='Scenario family',
                       loc = legend_settings[display_mode]["loc"],
                       bbox_to_anchor = legend_settings[display_mode]["bbox_to_anchor"],
                       ncol=1, frameon=True, fancybox=True, borderpad=1,
                       title_fontproperties=bold_font)

        ############## SAVE PARAMETERS ##############
        if save_state == "saved":
            suffix = f"_{rank_col}_Top{top_n}"
            fig.savefig(f"GSA_Delta{suffix}.png", dpi=600, bbox_inches="tight")
            fig.savefig(f"GSA_Delta{suffix}.pdf",             bbox_inches="tight")
            print(f"Saved GSA_Delta{suffix}.png/pdf")

        plt.show()

        df_display = selected[
            ['case', 'indicator', 'GSA name', 'GSA name cleaned', 'delta_min', 'delta_max', 'spread']
        ].reset_index(drop=True)
        styled_html = df_display.style.set_table_styles(
            [{
                'selector': 'th.col2, td.col2',
                'props': [('min-width', '300px')]
            }]
        ).to_html()
        display(HTML(styled_html))

############## WIRE EVERYTHING TOGETHER ##############
interactive_output = widgets.interactive_output(
    update_all,
    {
        'view_mode'   : view_selector,
        'metric'      : metric_toggle,
        'top_n'       : top_n_slider,
        'save_state'  : savefig,
        'display_mode': display_mode_toggle
    }
)

display(output)
update_all(view_selector.value,
           metric_toggle.value,
           top_n_slider.value,
           savefig.value,
           display_mode_toggle.value)

Accordion(children=(VBox(children=(HTML(value='<b>Select Ordering, Metric & Threshold</b>'), Dropdown(descript…

Output()